In [26]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorchcv.model_provider import get_model

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
cache = "/scratch/doa240/.cache/datasets"
train_data = torchvision.datasets.CIFAR10(root=cache, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)

Files already downloaded and verified


In [4]:
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [40]:
model = get_model("alexnet", pretrained=False)
model.eval()

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [41]:
import torch.nn as nn
model.output.fc2.fc = nn.Linear(4096,1024)
model.output.fc3 = nn.Linear(1024,10)
model.eval()

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [43]:
model.to(device)

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [44]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [45]:
import time

In [46]:
for epoch in range(7):  # loop over the dataset multiple times

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        #Time
        end_time = time.time()
        time_taken = end_time - start_time

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            print('Time:',time_taken)
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,  2000] loss: 1.865
Time: 19.444782972335815
[1,  4000] loss: 1.556
Time: 37.453460693359375
[1,  6000] loss: 1.426
Time: 55.52436184883118
[1,  8000] loss: 1.290
Time: 73.55392599105835
[1, 10000] loss: 1.197
Time: 91.59856486320496
[1, 12000] loss: 1.120
Time: 109.67991662025452
[2,  2000] loss: 0.980
Time: 18.788944959640503
[2,  4000] loss: 0.941
Time: 37.33146643638611
[2,  6000] loss: 0.898
Time: 55.86192464828491
[2,  8000] loss: 0.859
Time: 74.42028546333313
[2, 10000] loss: 0.841
Time: 93.04849672317505
[2, 12000] loss: 0.831
Time: 111.67383933067322
[3,  2000] loss: 0.654
Time: 19.06761074066162
[3,  4000] loss: 0.640
Time: 38.60707759857178
[3,  6000] loss: 0.662
Time: 58.14228892326355
[3,  8000] loss: 0.648
Time: 76.97777938842773
[3, 10000] loss: 0.643
Time: 95.56450247764587
[3, 12000] loss: 0.639
Time: 114.14690685272217
[4,  2000] loss: 0.437
Time: 18.876917600631714
[4,  4000] loss: 0.440
Time: 38.29993748664856
[4,  6000] loss: 0.486
Time: 57.814226150512695
[4,  

In [47]:
test_data = torchvision.datasets.CIFAR10(root=cache, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

Files already downloaded and verified


In [48]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 77.32 %


In [49]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Airplane : 82 %
Accuracy of   Car : 87 %
Accuracy of  Bird : 67 %
Accuracy of   Cat : 53 %
Accuracy of  Deer : 75 %
Accuracy of   Dog : 73 %
Accuracy of  Frog : 83 %
Accuracy of Horse : 79 %
Accuracy of  Ship : 82 %
Accuracy of Truck : 88 %


In [15]:
# #Verifying average accuracy of the network
# avg = 0
# for i in range(10):
#   temp = (100 * class_correct[i] / class_total[i])
#   avg = avg + temp
# avg = avg/10
# print('Average accuracy = ', avg)  